# Cryptography (CC4017) -- Week 10 - Extra 

## Q1: Man-in-the-middle attack

Implement a prototype that demonstrates how a Man-in-the-Middle attack can occur in a standardunauthenticated Diffie-Hellman key exchange. As usual, this happens between usual suspects Alice and Bob. In appendix, you can find four files:

- config_alice and config_bob are configuration files that are used by Alice and Bob to know who to talk to.

- alice.py establishes a connection with Bob (hopefully), and performs the Diffie-Hellman key exchange.
    
- bob.py mirrors the behavior of Alice.
    
The goal of the work is to design the man-in-the-middle adversary code:mitm.py. Your code must convince Alice and Bob to instead talk to him, and perform a key exchange with him. Your attack must not change the source code of Alice or Bob. Your attack is successful if Alice and Bob are not agreeing on the same secret, and the secrets they have agreed to are both known to the adversary.
    
**Suggestion:** Start by analysing the code for Alice and Bob, what are they using to communicate? How can we subvert this mechanism to be more. . . convenient?
    
To facilitate communication, this code usespwntools(reference). It is not mandatory to use this, butthe library considerably facilitates communication

In [ ]:
from pwn import *

# Configuration
hostAlice = "localhost"
portAlice = 5075  # Alice's port
hostBob = "localhost"
portBob = 5076  # Bob's port

# Diffie-Hellman parameters
g = 2
p = 7853799659

# Start listening for Alice
l_alice = listen(portAlice)
l_alice.wait_for_connection()
print("Connected to Alice")

# Start connecting to Bob
r_bob = remote(hostBob, portBob)
print("Connected to Bob")

# Intercept Alice's GX
gx_from_alice = int.from_bytes(l_alice.recvline()[:-1], "little")
print("Intercepted GX from Alice:", gx_from_alice)

# Send our own GX to Bob
x_mitm = random.randint(1, p)
gx_from_mitm_to_bob = pow(g, x_mitm, p)
r_bob.sendline(gx_from_mitm_to_bob.to_bytes(8, "little"))
print("Sent GX to Bob:", gx_from_mitm_to_bob)

# Intercept Bob's GY
gy_from_bob = int.from_bytes(r_bob.recvline()[:-1], "little")
print("Intercepted GY from Bob:", gy_from_bob)

# Send our own GY to Alice
y_mitm = random.randint(1, p)
gy_from_mitm_to_alice = pow(g, y_mitm, p)
l_alice.sendline(gy_from_mitm_to_alice.to_bytes(8, "little"))
print("Sent GY to Alice:", gy_from_mitm_to_alice)

# Calculate secrets
shared_secret_with_alice = pow(gy_from_mitm_to_alice, x_mitm, p)
shared_secret_with_bob = pow(gx_from_mitm_to_bob, y_mitm, p)

print("Shared secret with Alice:", shared_secret_with_alice)
print("Shared secret with Bob:", shared_secret_with_bob)

# Close connections
l_alice.close()
r_bob.close()


: 